In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd


def init_browser():
    #Windows
    executable_path = {'executable_path': 'chromedriver.exe'}
    return Browser('chrome', **executable_path, headless=False)



def data_scrape(url):
    browser = init_browser()
    browser.visit(url)

    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    browser.quit()
    
    return soup

result_dict = {}

## Collection of URL calls for scraping

This was used for my testing so I did not call the URL too many times.

In [2]:
print("Starting scape")

# article_listings = data_scrape("https://mars.nasa.gov/news/")
# mars_photo_scrap = data_scrape("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")
# mars_weather = data_scrape("https://twitter.com/marswxreport?lang=en")
# mars_facts = data_scrape("https://space-facts.com/mars/")
# mars_hires = data_scrape("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")
mars_hires2 = data_scrape("https://astrogeology.usgs.gov/maps/mars-viking-hemisphere-point-perspectives")

print("Done")

Starting scape
Done


## Gather the titles and teasers of the article page

In [34]:
# article_listings = data_scrape("https://mars.nasa.gov/news/")

titles = article_listings.find_all('div', class_='list_text')
subtitle = article_listings.find_all('div', class_='article_teaser_body')

article_range = range(len(titles))

title_list = []
teaser_list = []

for article in article_range:
    title_list.append(titles[article].a.text)
    teaser_list.append(subtitle[article].text)

    
first_article_title = title_list[0]
first_article_teaser = teaser_list[0]
# print(first_article_title)
# print(first_article_teaser)

articles = {'title': first_article_title, 'teaser': first_article_teaser}
result_dict['articles'] = articles

print(result_dict)

{'articles': {'title': "Six Things to Know About NASA's Opportunity Rover", 'teaser': "Opportunity's mission is complete. Here are highlights from its time on Mars."}, 'main photo': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17936_hires.jpg'}


## Gather Mars photo link

In [35]:
base_url = "https://www.jpl.nasa.gov"
search_value = "/spaceimages/?search=&category=Mars"
url = base_url + search_value

# mars_photo_scrap = data_scrape(url)

main_photo_section = mars_photo_scrap.find_all('a', class_='button', id='full_image')


# soup.find('div').attrs
photo_link = base_url + main_photo_section[0].attrs['data-fancybox-href']
large_photo_link = photo_link.replace('medium','large')
large_photo_link = large_photo_link.replace('ip','hires')

# print(photo_link)
# print(large_photo_link)


result_dict['main photo'] = large_photo_link
print(result_dict)

{'articles': {'title': "Six Things to Know About NASA's Opportunity Rover", 'teaser': "Opportunity's mission is complete. Here are highlights from its time on Mars."}, 'main photo': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17936_hires.jpg'}


## Gather Weather data from twitter feed

In [38]:


mars_weather_text = mars_weather.find_all('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')

tweet_range = range(len(mars_weather_text))

# we only want the most recent weather data so break from loop as soon as the first tweet that has the text starting with 
# Sol is found
for tweet in tweet_range:
    if mars_weather_text[tweet].text[:3] == 'Sol':
        weather_string = mars_weather_text[tweet].text
        break

t = weather_string.find("pic.twitter")
weather_string = weather_string[:t]

result_dict['weather'] = weather_string

print(result_dict)
# print(weather_string)

# I thought it might be nice to parse out the weather data into seperate variables.
# This function parses a string based on the first comma found.  It sets a variable to the data before the comma
# and resets the source string to itself minus the ", " so it can be used for the next variable.  If there is no
# comma (t==-1) then it sets the result string = to the source_string
# I'm sure someone has built a more elegant solution to this I just wanted to make this function for the fun of it.
# I probably should redo this as a loop and fill a dictionary with each of the values to make the execution dryer
# def parse_comma(source_string):
#     t = source_string.find(",")
#     if t == -1:
#         t = len(source_string)
#     result_string = source_string[:t]
#     source_string = source_string[t+2:]
#     return source_string, result_string

# parse_string = weather_string
# parse_string, weather_date = parse_comma(parse_string)
# parse_string, weather_high = parse_comma(parse_string)
# parse_string, weather_low = parse_comma(parse_string)
# parse_string, weather_pressure = parse_comma(parse_string)
# parse_string, weather_daylight = parse_comma(parse_string)


# print(weather_string)
# print(weather_date)
# print(weather_high)
# print(weather_low)
# print(weather_pressure)
# print(weather_daylight)



{'articles': {'title': "Six Things to Know About NASA's Opportunity Rover", 'teaser': "Opportunity's mission is complete. Here are highlights from its time on Mars."}, 'main photo': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17936_hires.jpg', 'weather': 'Sol 2319 (2019-02-13), high -17C/1F, low -72C/-97F, pressure at 8.12 hPa, daylight 06:46-18:52'}


## Gather Mars Facts

In [41]:
mars_facts_title_soup = mars_facts.find_all('td', class_='column-1')
mars_facts_value_soup = mars_facts.find_all('td', class_='column-2')

table_range = range(len(mars_facts_title_soup))

mars_facts_titles = []
mars_facts_values = []
for row in table_range:
    mars_facts_titles.append(mars_facts_title_soup[row].text)
    mars_facts_values.append(mars_facts_value_soup[row].text)

mars_facts_table = pd.DataFrame({"title": mars_facts_titles,
                                "value": mars_facts_values})


mars_facts_table.to_html('templates\mars_facts.html', index=False)


with open('templates\mars_facts.html', 'r') as f:
        table_html = f.read()

result_dict['facts'] = table_html
print(result_dict)

{'articles': {'title': "Six Things to Know About NASA's Opportunity Rover", 'teaser': "Opportunity's mission is complete. Here are highlights from its time on Mars."}, 'main photo': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17936_hires.jpg', 'weather': 'Sol 2319 (2019-02-13), high -17C/1F, low -72C/-97F, pressure at 8.12 hPa, daylight 06:46-18:52', 'facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>title</th>\n      <th>value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km\\n</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km\\n</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit

## Gather Hi-Res Hemishere links

In [3]:
print(mars_hires2)

<!DOCTYPE html>
<html lang="en" xmlns="http://www.w3.org/1999/xhtml"><head>
<title>Mars Viking Hemisphere Point Perspectives | USGS Astrogeology Science Center</title>
<meta content="USGS Astrogeology Science Center works with NASA and space-science communities to integrate planetary maps, geology, and remote sensing for Mars, Venus, the Moon, Mercury, Asteroids, Titan, Io, and more." name="description"/>
<meta content="USGS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping,Mars" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<meta content="x61hXXVj7wtfBSNOPnTftajMsZ5yB2W-qRoyr7GtOKM" name="google-site-verification"/>
<!--<link rel="stylesheet" href="http://fonts.googleapis.com/css?family=Open+Sans:400italic,400,bold"/>-->
<link href="/css/main.css" media="screen" rel="styleshe

In [5]:
# print(mars_hires)
# mars_hires_soup = mars_hires.find('title')

# This site was down
# https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mar
# I found another way to get the tif file links
# https://astrogeology.usgs.gov/maps/mars-viking-hemisphere-point-perspectives
# once on this site I need to scrape the links from other pages
# once https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mar is back online 
# I intend to write code to check for 404.  When 404 I will use the code developed below to capture picture links
# Otherwise I will scrape the original link for the image links.

# if mars_hires_soup.text[:9] == 'Error 404':

base_url = "https://astrogeology.usgs.gov"

mars_hires2_soup = mars_hires2.find_all('a', class_="item")

pict_range = range(len(mars_hires2_soup))

pict_list = []
for pict in pict_range:
    # check to see if the href ends in _enhanced
    if mars_hires2_soup[pict].attrs['href'][-9:] == '_enhanced':
        # if it does scrape the new url for image links
        pict_html = data_scrape(base_url + mars_hires2_soup[pict].attrs['href'])
#         pict_soup = pict_html.find_all('dd')
        pict_soup = pict_html.find_all('div', class_='downloads')

        # add a dictionry containing the image title, the hi-res photo link and a thumb image link
        # the thumb image link is added so my site will still be able to display an image even if the hi-res is down
#         img_url_test = data_scrape(pict_soup[1].a.attrs['href'])
#         pict_list.append({'title': mars_hires2_soup[pict].h3.text[:-9], 'img_url': pict_soup[1].a.attrs['href'], 'tmb_url':\
#                          base_url + mars_hires2_soup[pict].img.attrs['src']})
        pict_list.append({'title': mars_hires2_soup[pict].h3.text[:-9], 'img_url': pict_soup[0].a.attrs['href'], 'tmb_url':\
                         base_url + pict_soup[0].img.attrs['src']})

result_dict['hi-res'] = pict_list
print(result_dict)

# Added a URL for the thumb images 'tmb_url'.  That way, if the tiff images are still ofline my website will still have 
# an image to display

    

{'hi-res': [{'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg', 'tmb_url': 'https://astrogeology.usgs.gov/cache/images/04085d99ec3713883a9a57f42be9c725_valles_marineris_enhanced.tif_thumb.png'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'tmb_url': 'https://astrogeology.usgs.gov/cache/images/aae41197e40d6d4f3ea557f8cfe51d15_syrtis_major_enhanced.tif_thumb.png'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'tmb_url': 'https://astrogeology.usgs.gov/cache/images/7677c0a006b83871b5a2f66985ab5857_schiaparelli_enhanced.tif_thumb.png'}, {'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'tmb_url': 'htt